<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

# Задание 3

## Классификация текстов

В этом задании вам предстоит попробовать несколько методов, используемых в задаче классификации, а также понять насколько хорошо модель понимает смысл слов и какие слова в примере влияют на результат.

In [1]:
import pandas as pd
import numpy as np
import torch
import nltk
from torchtext.legacy import datasets
from torchtext.legacy.data import Field, LabelField
from torchtext.legacy.data import BucketIterator

from torchtext.vocab import Vectors, GloVe

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
from tqdm.autonotebook import tqdm

В этом задании мы будем использовать библиотеку torchtext. Она довольна проста в использовании и поможет нам сконцентрироваться на задаче, а не на написании Dataloader-а.

In [2]:
TEXT = Field(sequential=True, lower=True, include_lengths=True)  # Поле текста
LABEL = LabelField(dtype=torch.float)  # Поле метки

In [3]:
SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

Датасет на котором мы будем проводить эксперементы это комментарии к фильмам из сайта IMDB.

In [4]:
train, test = datasets.IMDB.splits(TEXT, LABEL)  # загрузим датасет
train, valid = train.split(random_state=random.seed(SEED))  # разобьем на части

downloading aclImdb_v1.tar.gz


100%|██████████| 84.1M/84.1M [00:07<00:00, 11.2MB/s]


In [5]:
TEXT.build_vocab(train, max_size=1000)
LABEL.build_vocab(train)

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"

train_iter, valid_iter, test_iter = BucketIterator.splits(
    (train, valid, test), 
    batch_size = 64,
    sort_within_batch = True,
    device = device
)

## RNN

Для начала попробуем использовать рекурентные нейронные сети. На семинаре вы познакомились с GRU, вы можете также попробовать LSTM. Можно использовать для классификации как hidden_state, так и output последнего токена.

In [7]:
class RNNBaseline(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.rnn = nn.LSTM(
            input_size=embedding_dim, 
            hidden_size=hidden_dim, 
            num_layers=n_layers, 
            bidirectional=bidirectional, 
            dropout=dropout
        )  # YOUR CODE GOES HERE

        linear_dim = hidden_dim * (bidirectional + 1)
        self.fc = nn.Linear(linear_dim, output_dim)  # YOUR CODE GOES HERE
        
    def forward(self, text, text_lengths):
        #text = [sent len, batch size]
        embedded = self.embedding(text)
        #embedded = [sent len, batch size, emb dim]

        #pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)
        
        # cell arg for LSTM, remove for GRU
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        #unpack sequence
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)

        #output = [sent len, batch size, hid dim * num directions]
        #output over padding tokens are zero tensors
        
        #hidden = [num layers * num directions, batch size, hid dim]
        #cell = [num layers * num directions, batch size, hid dim]
        
        #concat the final forward direction (hidden[-2,:,:]) and backward direction (hidden[-1,:,:]) hidden layers
        #and apply dropout
        
        output = output[-1, :, :]  # YOUR CODE GOES HERE
                
        #hidden = [batch size, hid dim * num directions] or [batch_size, hid dim * num directions]
        return self.fc(output)

Поиграйтесь с гиперпараметрами

In [8]:
vocab_size = len(TEXT.vocab)
emb_dim = 100
hidden_dim = 256
output_dim = 1
n_layers = 2
bidirectional = True
dropout = 0.2
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
patience=3

In [9]:
model = RNNBaseline(
    vocab_size=vocab_size,
    embedding_dim=emb_dim,
    hidden_dim=hidden_dim,
    output_dim=output_dim,
    n_layers=n_layers,
    bidirectional=bidirectional,
    dropout=dropout,
    pad_idx=PAD_IDX
)

In [10]:
model = model.to(device)

In [11]:
opt = torch.optim.Adam(model.parameters(), lr=0.001)
loss_func = nn.BCEWithLogitsLoss()

max_epochs = 20

Обучите сетку! Используйте любые вам удобные инструменты, Catalyst, PyTorch Lightning или свои велосипеды.

In [12]:
min_loss = np.inf

cur_patience = 0

for epoch in range(1, max_epochs + 1):
    train_loss = 0.0
    true_pred, count_pred = 0, 0
    model.train()
    pbar = tqdm(enumerate(train_iter), total=len(train_iter), leave=False)
    pbar.set_description(f"Epoch {epoch}")
    for it, batch in pbar:
        #YOUR CODE GOES HERE
        opt.zero_grad()
        seq, lengths = batch.text
        pred = model(seq, lengths.cpu())
        loss = loss_func(pred, batch.label.reshape(-1, 1))
        loss.backward()
        train_loss += loss.item()
        opt.step()

        true_pred += (torch.sigmoid(pred) >= 0.5).sum().item()
        count_pred += pred.shape[0]

    train_acc = true_pred / count_pred
    train_loss /= len(train_iter)
    val_loss = 0.0
    true_pred, count_pred = 0, 0
    model.eval()
    pbar = tqdm(enumerate(valid_iter), total=len(valid_iter), leave=False)
    pbar.set_description(f"Epoch {epoch}")
    for it, batch in pbar:
        # YOUR CODE GOES HERE
        seq, lengths = batch.text
        pred = model(seq, lengths.cpu())

        loss = loss_func(pred, batch.label.reshape(-1, 1))
        val_loss += loss.item()

        true_pred += (torch.sigmoid(pred) >= 0.5).sum().item()
        count_pred += pred.shape[0]

    val_acc = true_pred / count_pred
    val_loss /= len(valid_iter)
    if val_loss < min_loss:
        min_loss = val_loss
        best_model = model.state_dict()
    else:
        cur_patience += 1
        if cur_patience == patience:
            cur_patience = 0
            break
    
    print('Epoch: {}, Training Loss: {}, Train acc: {}, Validation Loss: {}, Validation acc: {}'.format(epoch, train_loss, train_acc, val_loss, val_acc))
model.load_state_dict(best_model)

  0%|          | 0/274 [00:00<?, ?it/s]

  0%|          | 0/118 [00:00<?, ?it/s]

Epoch: 1, Training Loss: 0.6932002392998577, Train acc: 0.9212571428571429, Validation Loss: 0.6916943980475604, Validation acc: 0.9349333333333333


  0%|          | 0/274 [00:00<?, ?it/s]

  0%|          | 0/118 [00:00<?, ?it/s]

...

  0%|          | 0/118 [00:00<?, ?it/s]

Epoch: 11, Training Loss: 0.6398576341406272, Train acc: 0.7213714285714286, Validation Loss: 0.640745297579442, Validation acc: 0.9081333333333333


  0%|          | 0/274 [00:00<?, ?it/s]

  0%|          | 0/118 [00:00<?, ?it/s]

<All keys matched successfully>

Посчитайте f1-score вашего классификатора на тестовом датасете.

**Ответ**:

In [13]:
from sklearn.metrics import f1_score

In [19]:
model.eval()
test_pred = []
test_true = [] 
pbar = tqdm(enumerate(test_iter), total=len(test_iter), leave=False)
pbar.set_description(f"Epoch {epoch}")
for it, batch in pbar:
    # YOUR CODE GOES HERE
    seq, lengths = batch.text
    pred = model(seq, lengths.cpu())
    batch_test_pred = torch.sigmoid(pred).cpu().reshape(-1).tolist()
    batch_test_true = batch.label.cpu().tolist()
    test_pred.extend(batch_test_pred)
    test_true.extend(batch_test_true)

f1_score(np.array(test_true, dtype=int), np.array(test_pred) >= 0.5)

0.6626917392855323

## CNN

![](https://www.researchgate.net/publication/333752473/figure/fig1/AS:769346934673412@1560438011375/Standard-CNN-on-text-classification.png)

Для классификации текстов также часто используют сверточные нейронные сети. Идея в том, что как правило сентимент содержат словосочетания из двух-трех слов, например "очень хороший фильм" или "невероятная скука". Проходясь сверткой по этим словам мы получим какой-то большой скор и выхватим его с помощью MaxPool. Далее идет обычная полносвязная сетка. Важный момент: свертки применяются не последовательно, а параллельно. Давайте попробуем!

In [20]:
TEXT = Field(sequential=True, lower=True, batch_first=True)  # batch_first тк мы используем conv  
LABEL = LabelField(batch_first=True, dtype=torch.float)

train, tst = datasets.IMDB.splits(TEXT, LABEL)
trn, vld = train.split(random_state=random.seed(SEED))

TEXT.build_vocab(trn)
LABEL.build_vocab(trn)

device = "cuda" if torch.cuda.is_available() else "cpu"

In [21]:
train_iter, valid_iter, test_iter = BucketIterator.splits(
        (trn, vld, tst),
        batch_sizes=(128, 256, 256),
        sort=False,
        sort_key= lambda x: len(x.src),
        sort_within_batch=False,
        device=device,
        repeat=False,
)

Вы можете использовать Conv2d с `in_channels=1, kernel_size=(kernel_sizes[0], emb_dim))` или Conv1d c `in_channels=emb_dim, kernel_size=kernel_size[0]`. Но хорошенько подумайте над shape в обоих случаях.

In [22]:
class CNN(nn.Module):
    def __init__(
        self,
        vocab_size,
        emb_dim,
        out_channels,
        kernel_sizes: list,
        dropout=0.5,
    ):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, emb_dim)
        self.conv_0 = nn.Conv1d(in_channels=emb_dim, 
                                out_channels=out_channels, 
                                kernel_size=kernel_sizes[0])  # YOUR CODE GOES HERE
        
        self.conv_1 = nn.Conv1d(in_channels=emb_dim, 
                                out_channels=out_channels, 
                                kernel_size=kernel_sizes[1])  # YOUR CODE GOES HERE
        
        self.conv_2 = nn.Conv1d(in_channels=emb_dim, 
                                out_channels=out_channels, 
                                kernel_size=kernel_sizes[2])  # YOUR CODE GOES HERE
        
        self.fc = nn.Linear(len(kernel_sizes) * out_channels, 1)
        
        self.dropout = nn.Dropout(dropout)
        
        
    def forward(self, text):
        
        embedded = self.embedding(text)
        
        embedded = embedded.permute(0, 2, 1)  # may be reshape here
        
        conved_0 = F.relu(self.conv_0(embedded))  # may be reshape here
        conved_1 = F.relu(self.conv_1(embedded))  # may be reshape here
        conved_2 = F.relu(self.conv_2(embedded))  # may be reshape here
        
        pooled_0 = F.max_pool1d(conved_0, conved_0.shape[2]).squeeze(2)
        pooled_1 = F.max_pool1d(conved_1, conved_1.shape[2]).squeeze(2)
        pooled_2 = F.max_pool1d(conved_2, conved_2.shape[2]).squeeze(2)
        
        cat = self.dropout(torch.cat((pooled_0, pooled_1, pooled_2), dim=1))
            
        return self.fc(cat)

In [23]:
kernel_sizes = [3, 4, 5]
vocab_size = len(TEXT.vocab)
out_channels=64
dropout = 0.5
dim = 300

model = CNN(vocab_size=vocab_size, emb_dim=dim, out_channels=out_channels,
            kernel_sizes=kernel_sizes, dropout=dropout)

In [24]:
model.to(device)

CNN(
  (embedding): Embedding(202223, 300)
  (conv_0): Conv1d(300, 64, kernel_size=(3,), stride=(1,))
  (conv_1): Conv1d(300, 64, kernel_size=(4,), stride=(1,))
  (conv_2): Conv1d(300, 64, kernel_size=(5,), stride=(1,))
  (fc): Linear(in_features=192, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [25]:
opt = torch.optim.Adam(model.parameters())
loss_func = nn.BCEWithLogitsLoss()

In [26]:
max_epochs = 30

Обучите!

In [27]:
min_loss = np.inf

cur_patience = 0

for epoch in range(1, max_epochs + 1):
    train_loss = 0.0
    model.train()
    pbar = tqdm(enumerate(train_iter), total=len(train_iter), leave=False)
    pbar.set_description(f"Epoch {epoch}")
    for it, batch in pbar: 
        opt.zero_grad()
        seq = batch.text
        pred = model(seq)
        loss = loss_func(pred, batch.label.reshape(-1, 1))
        loss.backward()
        train_loss += loss.item()
        opt.step()

        # true_pred += (torch.sigmoid(pred) >= 0.5).sum().item()
        # count_pred += pred.shape[0]

    train_loss /= len(train_iter)
    val_loss = 0.0
    model.eval()
    pbar = tqdm(enumerate(valid_iter), total=len(valid_iter), leave=False)
    pbar.set_description(f"Epoch {epoch}")
    for it, batch in pbar:
        seq = batch.text
        pred = model(seq)

        loss = loss_func(pred, batch.label.reshape(-1, 1))
        val_loss += loss.item()

        # true_pred += (torch.sigmoid(pred) >= 0.5).sum().item()
        # count_pred += pred.shape[0]

    val_loss /= len(valid_iter)
    if val_loss < min_loss:
        min_loss = val_loss
        best_model = model.state_dict()
    else:
        cur_patience += 1
        if cur_patience == patience:
            cur_patience = 0
            break
    
    print('Epoch: {}, Training Loss: {}, Validation Loss: {}'.format(epoch, train_loss, val_loss))
model.load_state_dict(best_model)

  0%|          | 0/137 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

Epoch: 1, Training Loss: 0.6637721892691006, Validation Loss: 0.49574731787045795


  0%|          | 0/137 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

...

  0%|          | 0/30 [00:00<?, ?it/s]

Epoch: 8, Training Loss: 0.14970555958630394, Validation Loss: 0.3293663869301478


  0%|          | 0/137 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

<All keys matched successfully>

Посчитайте f1-score вашего классификатора.

**Ответ**:

In [28]:
model.eval()
test_pred = []
test_true = [] 
pbar = tqdm(enumerate(test_iter), total=len(test_iter), leave=False)
pbar.set_description(f"Epoch {epoch}")
for it, batch in pbar:
    # YOUR CODE GOES HERE
    seq = batch.text
    pred = model(seq)
    batch_test_pred = torch.sigmoid(pred).cpu().reshape(-1).tolist()
    batch_test_true = batch.label.cpu().tolist()
    test_pred.extend(batch_test_pred)
    test_true.extend(batch_test_true)

f1_score(np.array(test_true, dtype=int), np.array(test_pred) >= 0.5)

  0%|          | 0/98 [00:00<?, ?it/s]

0.8541633370624899

## Интерпретируемость

Посмотрим, куда смотрит наша модель. Достаточно запустить код ниже.

In [29]:
!pip install -q captum

     |████████████████████████████████| 1.4 MB 4.3 MB/s 


In [30]:
from captum.attr import LayerIntegratedGradients, TokenReferenceBase, visualization

PAD_IND = TEXT.vocab.stoi['pad']

token_reference = TokenReferenceBase(reference_token_idx=PAD_IND)
lig = LayerIntegratedGradients(model, model.embedding)

In [31]:
def forward_with_softmax(inp):
    logits = model(inp)
    return torch.softmax(logits, 0)[0][1]

def forward_with_sigmoid(input):
    return torch.sigmoid(model(input))


# accumalate couple samples in this array for visualization purposes
vis_data_records_ig = []

def interpret_sentence(model, sentence, min_len = 7, label = 0):
    model.eval()
    text = [tok for tok in TEXT.tokenize(sentence)]
    if len(text) < min_len:
        text += ['pad'] * (min_len - len(text))
    indexed = [TEXT.vocab.stoi[t] for t in text]

    model.zero_grad()

    input_indices = torch.tensor(indexed, device=device)
    input_indices = input_indices.unsqueeze(0)

    # input_indices dim: [sequence_length]
    seq_length = min_len

    # predict
    pred = forward_with_sigmoid(input_indices).item()
    pred_ind = round(pred)

    # generate reference indices for each sample
    reference_indices = token_reference.generate_reference(seq_length, device=device).unsqueeze(0)

    # compute attributions and approximation delta using layer integrated gradients
    attributions_ig, delta = lig.attribute(input_indices, reference_indices, \
                                           n_steps=5000, return_convergence_delta=True)

    print('pred: ', LABEL.vocab.itos[pred_ind], '(', '%.2f'%pred, ')', ', delta: ', abs(delta))

    add_attributions_to_visualizer(attributions_ig, text, pred, pred_ind, label, delta, vis_data_records_ig)
    
def add_attributions_to_visualizer(attributions, text, pred, pred_ind, label, delta, vis_data_records):
    attributions = attributions.sum(dim=2).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    attributions = attributions.cpu().detach().numpy()

    # storing couple samples in an array for visualization purposes
    vis_data_records.append(visualization.VisualizationDataRecord(
                            attributions,
                            pred,
                            LABEL.vocab.itos[pred_ind],
                            LABEL.vocab.itos[label],
                            LABEL.vocab.itos[1],
                            attributions.sum(),       
                            text,
                            delta))

In [32]:
interpret_sentence(model, 'It was a fantastic performance !', label=1)
interpret_sentence(model, 'Best film ever', label=1)
interpret_sentence(model, 'Such a great show!', label=1)
interpret_sentence(model, 'It was a horrible movie', label=0)
interpret_sentence(model, 'I\'ve never watched something as bad', label=0)
interpret_sentence(model, 'It is a disgusting movie!', label=0)

pred:  pos ( 0.89 ) , delta:  tensor([8.6025e-05], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.02 ) , delta:  tensor([1.4645e-05], device='cuda:0', dtype=torch.float64)
pred:  pos ( 0.83 ) , delta:  tensor([8.5923e-05], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.03 ) , delta:  tensor([2.4782e-05], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.04 ) , delta:  tensor([4.1495e-05], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.08 ) , delta:  tensor([0.0002], device='cuda:0', dtype=torch.float64)


Попробуйте добавить свои примеры!

In [33]:
print('Visualize attributions based on Integrated Gradients')
visualization.visualize_text(vis_data_records_ig)

Visualize attributions based on Integrated Gradients


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
pos,pos (0.89),pos,1.65,It was a fantastic performance ! pad
pos,neg (0.02),pos,0.87,Best film ever pad pad pad pad
pos,pos (0.83),pos,1.42,Such a great show! pad pad pad
neg,neg (0.03),pos,0.35,It was a horrible movie pad pad
neg,neg (0.04),pos,0.38,I've never watched something as bad pad
neg,neg (0.08),pos,0.81,It is a disgusting movie! pad pad


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
pos,pos (0.89),pos,1.65,It was a fantastic performance ! pad
pos,neg (0.02),pos,0.87,Best film ever pad pad pad pad
pos,pos (0.83),pos,1.42,Such a great show! pad pad pad
neg,neg (0.03),pos,0.35,It was a horrible movie pad pad
neg,neg (0.04),pos,0.38,I've never watched something as bad pad
neg,neg (0.08),pos,0.81,It is a disgusting movie! pad pad


## Эмбеддинги слов

Вы ведь не забыли, как мы можем применить знания о word2vec и GloVe. Давайте попробуем!

In [34]:
glove = GloVe()

.vector_cache/glove.840B.300d.zip: 2.18GB [06:53, 5.26MB/s]                            
100%|█████████▉| 2196016/2196017 [04:56<00:00, 7397.99it/s]


In [35]:
TEXT.build_vocab(train, vectors=glove)# YOUR CODE GOES HERE
# подсказка: один из импортов пока не использовался, быть может он нужен в строке выше :)
LABEL.build_vocab(train)

word_embeddings = TEXT.vocab.vectors

kernel_sizes = [3, 4, 5]
vocab_size = len(TEXT.vocab)
dropout = 0.5
dim = 300

In [36]:
train, test = datasets.IMDB.splits(TEXT, LABEL)
train, valid = train.split(random_state=random.seed(SEED))

device = "cuda" if torch.cuda.is_available() else "cpu"
train_iter, valid_iter, test_iter = BucketIterator.splits(
        (train, valid, test),
        batch_sizes=(128, 256, 256),
        sort=False,
        sort_key= lambda x: len(x.src),
        sort_within_batch=False,
        device=device,
        repeat=False,
)

In [37]:
model = CNN(vocab_size=vocab_size, emb_dim=dim, out_channels=64,
            kernel_sizes=kernel_sizes, dropout=dropout)

word_embeddings = TEXT.vocab.vectors

prev_shape = model.embedding.weight.shape

with torch.no_grad():
    model.embedding.weight = nn.Parameter(word_embeddings)  # инициализируйте эмбэдинги

assert prev_shape == model.embedding.weight.shape
model.to(device)

opt = torch.optim.Adam(model.parameters())

In [38]:
def freeze_embeddings(model, req_grad=False):
    embeddings = model.embedding
    for c_p in embeddings.parameters():
        c_p.requires_grad = req_grad

Вы знаете, что делать.

In [39]:
min_loss = np.inf

cur_patience = 0
freeze_embeddings(model, req_grad=False)
for epoch in range(1, max_epochs + 1):
    train_loss = 0.0
    model.train()
    pbar = tqdm(enumerate(train_iter), total=len(train_iter), leave=False)
    pbar.set_description(f"Epoch {epoch}")
    for it, batch in pbar: 
        #YOUR CODE GOES HERE
        opt.zero_grad()
        seq = batch.text
        pred = model(seq)
        loss = loss_func(pred, batch.label.reshape(-1, 1))
        loss.backward()
        train_loss += loss.item()
        opt.step()

    train_loss /= len(train_iter)
    val_loss = 0.0
    model.eval()
    pbar = tqdm(enumerate(valid_iter), total=len(valid_iter), leave=False)
    pbar.set_description(f"Epoch {epoch}")
    for it, batch in pbar:
        # YOUR CODE GOES HERE
        seq = batch.text
        pred = model(seq)

        loss = loss_func(pred, batch.label.reshape(-1, 1))
        val_loss += loss.item()

    val_loss /= len(valid_iter)
    if val_loss < min_loss:
        min_loss = val_loss
        best_model = model.state_dict()
    else:
        cur_patience += 1
        if cur_patience == patience:
            cur_patience = 0
            break
    
    print('Epoch: {}, Training Loss: {}, Validation Loss: {}'.format(epoch, train_loss, val_loss))
model.load_state_dict(best_model)

  0%|          | 0/137 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

Epoch: 1, Training Loss: 0.5091834749183516, Validation Loss: 0.36969223121802014


  0%|          | 0/137 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

...

  0%|          | 0/30 [00:00<?, ?it/s]

Epoch: 9, Training Loss: 0.15577936493349773, Validation Loss: 0.2853366782267888


  0%|          | 0/137 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

<All keys matched successfully>

Посчитайте f1-score вашего классификатора.

**Ответ**:

In [40]:
model.eval()
test_pred = []
test_true = [] 
pbar = tqdm(enumerate(test_iter), total=len(test_iter), leave=False)
pbar.set_description(f"Epoch {epoch}")
for it, batch in pbar:
    # YOUR CODE GOES HERE
    seq = batch.text
    pred = model(seq)
    batch_test_pred = torch.sigmoid(pred).cpu().reshape(-1).tolist()
    batch_test_true = batch.label.cpu().tolist()
    test_pred.extend(batch_test_pred)
    test_true.extend(batch_test_true)

f1_score(np.array(test_true, dtype=int), np.array(test_pred) >= 0.5)

  0%|          | 0/98 [00:00<?, ?it/s]

0.8787719227186196

Проверим насколько все хорошо!

In [41]:
PAD_IND = TEXT.vocab.stoi['pad']

token_reference = TokenReferenceBase(reference_token_idx=PAD_IND)
lig = LayerIntegratedGradients(model, model.embedding)
vis_data_records_ig = []

interpret_sentence(model, 'It was a fantastic performance !', label=1)
interpret_sentence(model, 'Best film ever', label=1)
interpret_sentence(model, 'Such a great show!', label=1)
interpret_sentence(model, 'It was a horrible movie', label=0)
interpret_sentence(model, 'I\'ve never watched something as bad', label=0)
interpret_sentence(model, 'It is a disgusting movie!', label=0)

pred:  pos ( 0.97 ) , delta:  tensor([4.3423e-05], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.30 ) , delta:  tensor([3.5609e-05], device='cuda:0', dtype=torch.float64)
pred:  pos ( 0.57 ) , delta:  tensor([1.0598e-05], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.03 ) , delta:  tensor([3.7740e-05], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.34 ) , delta:  tensor([8.8939e-05], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.02 ) , delta:  tensor([5.0243e-05], device='cuda:0', dtype=torch.float64)


In [42]:
print('Visualize attributions based on Integrated Gradients')
visualization.visualize_text(vis_data_records_ig)

Visualize attributions based on Integrated Gradients


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
pos,pos (0.97),pos,1.60,It was a fantastic performance ! pad
pos,neg (0.30),pos,1.13,Best film ever pad pad pad pad
pos,pos (0.57),pos,1.40,Such a great show! pad pad pad
neg,neg (0.03),pos,-0.58,It was a horrible movie pad pad
neg,neg (0.34),pos,0.53,I've never watched something as bad pad
neg,neg (0.02),pos,-0.61,It is a disgusting movie! pad pad


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
pos,pos (0.97),pos,1.60,It was a fantastic performance ! pad
pos,neg (0.30),pos,1.13,Best film ever pad pad pad pad
pos,pos (0.57),pos,1.40,Such a great show! pad pad pad
neg,neg (0.03),pos,-0.58,It was a horrible movie pad pad
neg,neg (0.34),pos,0.53,I've never watched something as bad pad
neg,neg (0.02),pos,-0.61,It is a disgusting movie! pad pad
